In [1]:
!pip install '../input/birdmel/noisereduce-1.1.0/noisereduce-1.1.0'
!pip install '../input/birdmel/Keras_Applications-1.0.8-py3-none-any.whl'
!pip install '../input/birdmel/efficientnet-1.1.0/efficientnet-1.1.0'
!pip install '../input/birdmel/tensorflow_addons-0.11.1-cp37-cp37m-manylinux2010_x86_64.whl'

Processing /kaggle/input/birdmel/noisereduce-1.1.0/noisereduce-1.1.0
  Created wheel for noisereduce: filename=noisereduce-1.1.0-py3-none-any.whl size=7608 sha256=38074fde7f0226330f5e89d423d7f2a9d2034e63f31b8b76ea384b453a09d0c3
  Stored in directory: /root/.cache/pip/wheels/49/8f/7b/bf5d8c00277dd5beab590b95f4e803aca73f99a7640bee9c71
Successfully built noisereduce
Processing /kaggle/input/birdmel/Keras_Applications-1.0.8-py3-none-any.whl
Processing /kaggle/input/birdmel/efficientnet-1.1.0/efficientnet-1.1.0
  Created wheel for efficientnet: filename=efficientnet-1.1.0-py3-none-any.whl size=14141 sha256=2a16eec23642a291b89dbef7969ef0d172d128f18bcad4cbe3e7c8b4373dd4f9
  Stored in directory: /root/.cache/pip/wheels/fa/91/d3/2401dc7a5a531245ffc84b63cd11d6594f132c41a1382ec230
Successfully built efficientnet
Processing /kaggle/input/birdmel/tensorflow_addons-0.11.1-cp37-cp37m-manylinux2010_x86_64.whl
  Attempting uninstall: tensorflow-addons
    Found existing installation: tensorflow-addons 

In [2]:
import numpy as np 
import pandas as pd

import tensorflow as tf

import librosa
import noisereduce as no
import cv2
import matplotlib.pyplot as plt

import efficientnet.tfkeras as efn
import tensorflow_addons as tfa

/opt/conda/lib/python3.7/site-packages/noisereduce/noisereduce.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Load a pretrained EfficientNetB5 model

In [3]:
batch_size = 16
img_size = (256, 5150)
seed = 1
storage_dir = '../input/birdmel/train_img_split/train_img_split'

In [4]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    storage_dir,
    validation_split=0.1,
    subset="validation",
    seed=seed,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical'
)

Found 50303 files belonging to 264 classes.
Using 5030 files for validation.


In [5]:
class_names = np.array(val_ds.class_names)
class_num = len(val_ds.class_names)

In [6]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./127.5, offset=-1)

def resize(image, height=256, width=256):
    return tf.image.resize_with_crop_or_pad(image, target_height=height, target_width=width)

def augment_image_test(image):
    image = resize(image, 256, 216)
    image = resize(image)
    image = normalization_layer(image)

    return image

def prepare(ds):
    return ds.map(lambda x, y: (augment_image_test(x), y))

In [7]:
net = tf.keras.models.load_model('../input/birdmel/efficientnet_b5_tf_2_3_0.h5')

## Prediction

In [8]:
fmin = 300
def get_melspectrogram_db(file_path):
    n_fft = 2048
    hop_length = 512   
    n_mels = 256

    signal, sr = librosa.load(file_path)
    
    signal = no.reduce_noise(audio_clip=signal, noise_clip=signal, verbose=False)

    spec = librosa.feature.melspectrogram(
        signal, 
        sr=sr, 
        n_fft=n_fft,
        hop_length=hop_length,
        n_mels=n_mels,
        fmin=fmin
    )
    
    return librosa.power_to_db(spec, ref=np.max)

In [9]:
def spec_to_image(spec, eps=1e-6):
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)
    spec_scaled = np.flip(spec_scaled, axis=0)

    return spec_scaled

In [10]:
def img_to_imgs(img):
    size = 216 # 5 seconds
    
    width = img.shape[1]
    split_num = int(np.floor(width/size))
    residual = width - split_num*size
    end_split_index = width - residual
    
    imgs = []
    if split_num > 0:
        imgs = np.array_split(img[:, :end_split_index, :], split_num, axis=1)

    imgs.append(img[:, end_split_index:, :])
    
    return imgs

In [11]:
def file_to_imgs(file):
    if file.endswith(".mp3"):
        img = spec_to_image(get_melspectrogram_db(file))
        img = cv2.merge((img,img,img))
        imgs = img_to_imgs(img)
        
        return imgs
    else:
        return []

In [12]:
def predict_img(model, class_names, img, threshold = 0.5):
    tensor = augment_image_test(img)
    batch = tf.expand_dims(tensor, 0)
    probas = net.predict(x=batch)[0]
    selected_class_names = class_names[probas > threshold]
    return list(selected_class_names).sort()

In [13]:
def predict_imgs(model, class_names, imgs, threshold = 0.5):
    preds = []
    
    for img in imgs:
        preds += predict(model, class_names, img, threshold)
    
    return list(set(preds)).sort()

In [14]:
def predict_df(df, audio_dir='../input/birdsong-recognition/test_audio/'):
    row_ids = []
    birds = []
    
    current_file_name = ''
    current_images = []
    for index, row in df.iterrows():
        row_id = row['row_id']
        filename = row['audio_id']
        
        if filename != current_file_name:
            current_file_name = filename
            current_images = file_to_imgs(audio_dir + filename + '.mp3')
            print(filename)
           
        site = row['site']
        if site == 'site_3':
            preds = predict_imgs(net, class_names, current_images)
        else:
            preds = predict_img(net, class_names, current_images.pop(0))
            
        row_ids.append(row_id)
        birds.append(preds)
        
    return pd.DataFrame({
        'row_id': row_ids,
        'birds': list(map(lambda x: 'nocall' if len(x) == 0 else ' '.join(x), birds))
    })

In [15]:
test_df = pd.read_csv('../input/birdsong-recognition/test.csv')

In [16]:
test_df

,site,row_id,seconds,audio_id
0,site_1,site_1_0a997dff022e3ad9744d4e7bbf923288_5,5,0a997dff022e3ad9744d4e7bbf923288
1,site_1,site_1_0a997dff022e3ad9744d4e7bbf923288_10,10,0a997dff022e3ad9744d4e7bbf923288
2,site_1,site_1_0a997dff022e3ad9744d4e7bbf923288_15,15,0a997dff022e3ad9744d4e7bbf923288


In [17]:
import warnings
warnings.filterwarnings('ignore')

try:
    sub_df = predict_df(sample_df)
except:
    sub_df = pd.read_csv('../input/birdsong-recognition/sample_submission.csv')

In [18]:
sub_df

,row_id,birds
0,site_1_0a997dff022e3ad9744d4e7bbf923288_5,nocall
1,site_1_0a997dff022e3ad9744d4e7bbf923288_10,nocall
2,site_1_0a997dff022e3ad9744d4e7bbf923288_15,nocall


In [19]:
sub_df.to_csv('submission.csv', index=False)